In [ ]:
!pip install yfinance


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf

# For time stamps
from datetime import datetime

In [ ]:
# The tech stocks we'll use for this analysis
# tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']
tech_list = ['TSLA']

# end = datetime.now()
end = datetime(2022, 1,31)
start = datetime(end.year - 2, end.month, end.day)

for stock in tech_list:
    globals()[stock] = yf.download(stock, start, end)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# company_list = [AAPL, GOOG, MSFT, AMZN]
# company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]
company_list = [TSLA]
company_name = ["TESLA"]
for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df = pd.concat(company_list, axis=0)
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume,company_name
Date,,,,,,,
2022-01-14,1019.880005,1052.000000,1013.380005,1049.609985,1049.609985,24308100,TESLA
2022-01-18,1026.609985,1070.790039,1016.059998,1030.510010,1030.510010,22247800,TESLA
2022-01-19,1041.709961,1054.670044,995.000000,995.650024,995.650024,25147500,TESLA
2022-01-20,1009.729980,1041.660034,994.000000,996.270020,996.270020,23496200,TESLA
2022-01-21,996.340027,1004.549988,940.500000,943.900024,943.900024,34472000,TESLA
2022-01-24,904.760010,933.510010,851.469971,930.000000,930.000000,50521900,TESLA
2022-01-25,914.200012,951.260010,903.210022,918.400024,918.400024,28865300,TESLA
2022-01-26,952.429993,987.690002,906.000000,937.409973,937.409973,34955800,TESLA
2022-01-27,933.359985,935.390015,829.000000,829.099976,829.099976,49036500,TESLA


In [ ]:
# ma_day = [10, 20, 50] #stock price moving average peroid

# for ma in ma_day:
#     for company in company_list:
#         column_name = f"MA for {ma} days"
#         company[column_name] = company['Adj Close'].rolling(ma).mean()

In [ ]:
# # We'll use pct_change to find the percent change for each day
# for company in company_list:
#     company['Daily Return'] = company['Adj Close'].pct_change()


In [ ]:
from pandas_datareader import data as pdr
yf.pdr_override()
#closing stock price
closing_df = pdr.get_data_yahoo(tech_list, start=start,end=end)['Adj Close']


# Make a new tech returns DataFrame
tech_rets = closing_df.pct_change()
rets = tech_rets.dropna()

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Get the stock quote
closing_df = pdr.get_data_yahoo(tech_list, start=start,end=end)['Adj Close']


[*********************100%***********************]  1 of 1 completed


In [ ]:
# Create a new dataframe with only the 'Close column 
#!!!!!
data = df.filter(["Open",	"High",	"Low",	"Close",	"Adj Close",	"Volume"])
data_close=df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(dataset) * .95 )) #get 90% of the data to train
training_data_len

479

In [ ]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1)) #MinMaxScaler最小最大值標準化
scaled_data = scaler.fit_transform(dataset)
scaler_close = MinMaxScaler(feature_range=(0,1)) #MinMaxScaler最小最大值標準化
scaled_data_close = scaler_close.fit_transform(data_close)


In [ ]:
# Create the training data set 
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []
#!!!!可以改window size
#!!!!改成60*6
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, :])
    y_train.append(train_data[i, 3])
       
# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
#!!!!改layer
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 6)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

419/419 [==============================] - 17s 34ms/step - loss: 0.0047


In [ ]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:,3]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, :])
    
# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 6 ))

# Get the models predicted price values 
predictions = model.predict(x_test)
predictions = scaler_close.inverse_transform(predictions)
#改用MAPE看看
# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

117.14409561740341

In [ ]:
# Plot the data
data = df.filter(['Close'])
train = data[:training_data_len]
valid = data[training_data_len:]
predictions
valid['Predictions'] = predictions
valid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Close,Predictions
Date,,
2021-12-23,1067.000000,878.943909
2021-12-27,1093.939941,893.955078
2021-12-28,1088.469971,918.425232
2021-12-29,1086.189941,944.369507
2021-12-30,1070.339966,966.386780
2021-12-31,1056.780029,981.468994
2022-01-03,1199.780029,990.496643
2022-01-04,1149.589966,1012.021362
2022-01-05,1088.119995,1032.092163


In [ ]:
import random
from sklearn import metrics
print( valid['Close'][1])
ud_valid=[]
ud_Prediction=[]
for i in range(1,len(valid)):
  if valid['Close'][i]>valid['Close'][i-1]:
    ud_valid.append(1)
  elif valid['Close'][i]<valid['Close'][i-1]:
    ud_valid.append(-1)
  else:
    ud_valid.append(0)
for i in range(1,len(valid)):
  if valid['Predictions'][i]>valid['Predictions'][i-1]:
    ud_Prediction.append(1)
  elif valid['Predictions'][i]<valid['Predictions'][i-1]:
    ud_Prediction.append(-1)
  else:
    ud_Prediction.append(0)
print(ud_Prediction)
print(ud_valid)

cf_matrix = metrics.confusion_matrix(ud_valid, ud_Prediction)
print(cf_matrix)


1093.93994140625
[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1]
[[8 7]
 [7 2]]


In [ ]:
#predict today's stock price
x_train=[]
x_train.append(scaled_data[len(scaled_data)-60:len(scaled_data), :])
x_train = np.array(x_train)
x_train=np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6 ))

predictions_t = model.predict(x_train)
predictions_t = scaler_close.inverse_transform(predictions_t)
print(predictions_t)

[[849.84094]]


In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.1 MB/s 
     |████████████████████████████████| 210 kB 56.5 MB/s 
     |████████████████████████████████| 81 kB 8.8 MB/s 
     |████████████████████████████████| 78 kB 6.9 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 146 kB 66.1 MB/s 
     |████████████████████████████████| 112 kB 58.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=44a4aff64ba57e4349b99bd2d005446ff4cd0de60bbd0fab86010f328e7f5538
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def objective(trial, X=X, y=y):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna.
    """
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000, 100),
        'eta': trial.suggest_float("eta", 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    reg = xgb.XGBRegressor(**params)
    reg.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)], eval_metric='rmse',
            verbose=False)
    return mean_squared_error(y_valid, reg.predict(X_valid), squared=False)

In [ ]:
%%time
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 10)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)